In [2]:
import numpy as np
import pandas as pd

In [220]:
data = pd.DataFrame(np.array([[1,69,1],[0,69,1],[1,67,0],[0,69,0],[2,56,2],[3,67,2],[1,67,3],[3,56,3]]))
data.columns = ['Player_#','Score','Tournament']

In [221]:
data

,Player_#,Score,Tournament
0,1,69,1
1,0,69,1
2,1,67,0
3,0,69,0
4,2,56,2
5,3,67,2
6,1,67,3
7,3,56,3


In [222]:
n_players = len(pd.unique(data['Player_#']))
n_tournaments = len(pd.unique(data.Tournament))

In [241]:
BETA = .5

In [242]:
A = np.zeros((n_players*n_tournaments,n_players*n_tournaments))
G = np.zeros((n_players*n_tournaments,n_players*n_tournaments))
for tournament in pd.unique(data.Tournament):
    subset = data[data.Tournament==tournament].sort_values('Score')
    for u,player in enumerate(pd.unique(subset['Player_#'])):
        arr = np.zeros(n_players)
        arr1 = np.zeros(n_players)
        #arr[pd.unique(subset['Player_#'])] = 1
        arr[pd.unique(subset['Player_#'])] = subset.Score.values / subset[subset['Player_#']==player].Score.values.astype(float) / len(subset)
        arr[pd.unique(subset['Player_#'])[u]] = 0
        arr1[pd.unique(subset['Player_#'])] = 1
        arr1[pd.unique(subset['Player_#'])[u]] = 0
        A[player + n_players*tournament,n_players*tournament:n_players*(tournament+1)] += arr
        G[player + n_players*tournament,n_players*tournament:n_players*(tournament+1)] += arr1
        for tournament2 in pd.unique(data.Tournament):
            if tournament==tournament2:
                continue
            else:
                A[player + n_players*tournament2,n_players*tournament:n_players*(tournament+1)] += BETA**(abs(tournament-tournament2)) * arr
                G[player + n_players*tournament2,n_players*tournament:n_players*(tournament+1)] += BETA**(abs(tournament-tournament2)) * arr1

In [243]:
V = np.matrix(np.nan_to_num(A/G))

In [244]:
alpha = .1

In [245]:
w_a = (np.linalg.inv(np.eye(n_players*n_tournaments) - alpha*A) * np.matrix(A).sum(1)).sum(1)

In [246]:
w_g = (np.linalg.inv(np.eye(n_players*n_tournaments) - alpha*G) * np.matrix(G).sum(1)).sum(1)

In [247]:
w_v = (np.linalg.inv(np.eye(n_players*n_tournaments) - alpha*V) * np.matrix(V).sum(1)).sum(1)

In [248]:
w_a/w_g

matrix([[ 0.44668501],
        [ 0.46094527],
        [ 0.55044604],
        [ 0.44979841],
        [ 0.4499194 ],
        [ 0.45183249],
        [ 0.55044604],
        [ 0.44979841],
        [ 0.4499194 ],
        [ 0.4325357 ],
        [ 0.55044604],
        [ 0.44979841],
        [ 0.4499194 ],
        [ 0.40787872],
        [ 0.55044604],
        [ 0.50002843]])